In [1]:
!pip install pymystem3

In [2]:
# !pip install pymorphy2
# !pip install pymorphy2-dicts-ru
# !pip install DAWG-Python

In [3]:
import numpy as np
import os
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from pymystem3 import Mystem
# from pymorphy2 import MorphAnalyzer
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from string import punctuation
from tqdm import tqdm

In [4]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/preprocessed-data/df_preprocessed.csv
/kaggle/input/texts-classification-iad-hse-intro-2020/test.csv
/kaggle/input/texts-classification-iad-hse-intro-2020/sample_submission.csv
/kaggle/input/texts-classification-iad-hse-intro-2020/train.csv


In [5]:
DATA_DIR = '/kaggle/input/'

# df_test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'), dtype={'itemid' : 'uint32'})
# sub = pd.read_csv(os.path.join(DATA_DIR, 'sample_submission.csv'), dtype={'itemid' : 'uint32', 'Category' : 'uint8'})

In [6]:
df_test = pd.read_csv(os.path.join(DATA_DIR, 'texts-classification-iad-hse-intro-2020/test.csv'), dtype={'itemid' : 'uint32'})

In [7]:
df_test

,title,description,itemid
0,Мастерка,Мастерка фирмы форвард. Белого цвета. В идеаль...,1778449823
1,Зимние сапоги,"Продаю зимние сапоги, в хорошем состоянии, все...",1677656962
2,Видеонаблюдение 8 камер,В комплект Atis AMD-2MIR-8kit входит: /\n1. Ку...,1758182804
3,Запчасти для GLE,Запчасти GLE,1689811299
4,Бластер nerf,Состояние 5+/\nПродаю потому что не нужен/\n18...,1804706240
...,...,...,...
1411343,Бетоносмеситель Россия бсэ-140 чугун,Бетономешалка российского производства - не Ки...,902146704
1411344,Разношрокие колеса R19 оригинал Porsche Panamera,Диски с резиной в отличном состоянии/\n /\nШин...,1799983289
1411345,Комбенз зимний размер,"Комбензон на овчине .Подойдет зима , весна, ос...",1177237748
1411346,Смартфон Honor 5C,В отличном состоянии. Использовался с защитным...,1793055786


In [8]:
df_test.isnull().sum()

title          0
description    1
itemid         0
dtype: int64

In [9]:
df_test.fillna('', inplace=True)

In [10]:
df_test.isnull().sum()

title          0
description    0
itemid         0
dtype: int64

In [11]:
%%time

df_test['title&description'] = df_test['title'].str[:] + ' ' + df_test['description'].str[:]
df_test.head()

CPU times: user 2.66 s, sys: 1.75 s, total: 4.4 s
Wall time: 4.41 s


,title,description,itemid,title&description
0,Мастерка,Мастерка фирмы форвард. Белого цвета. В идеаль...,1778449823,Мастерка Мастерка фирмы форвард. Белого цвета....
1,Зимние сапоги,"Продаю зимние сапоги, в хорошем состоянии, все...",1677656962,"Зимние сапоги Продаю зимние сапоги, в хорошем ..."
2,Видеонаблюдение 8 камер,В комплект Atis AMD-2MIR-8kit входит: /\n1. Ку...,1758182804,Видеонаблюдение 8 камер В комплект Atis AMD-2M...
3,Запчасти для GLE,Запчасти GLE,1689811299,Запчасти для GLE Запчасти GLE
4,Бластер nerf,Состояние 5+/\nПродаю потому что не нужен/\n18...,1804706240,Бластер nerf Состояние 5+/\nПродаю потому что ...


In [12]:
df_test = df_test.drop(['title','description'], axis=1)

In [13]:
df_test

,itemid,title&description
0,1778449823,Мастерка Мастерка фирмы форвард. Белого цвета....
1,1677656962,"Зимние сапоги Продаю зимние сапоги, в хорошем ..."
2,1758182804,Видеонаблюдение 8 камер В комплект Atis AMD-2M...
3,1689811299,Запчасти для GLE Запчасти GLE
4,1804706240,Бластер nerf Состояние 5+/\nПродаю потому что ...
...,...,...
1411343,902146704,Бетоносмеситель Россия бсэ-140 чугун Бетономеш...
1411344,1799983289,Разношрокие колеса R19 оригинал Porsche Paname...
1411345,1177237748,Комбенз зимний размер Комбензон на овчине .Под...
1411346,1793055786,Смартфон Honor 5C В отличном состоянии. Исполь...


In [14]:
df_test.shape

(1411348, 2)

In [15]:
%%time

df_test['title&description'] = df_test['title&description'].str.lower()
df_test['title&description'] = df_test['title&description'].str.replace(r"http\S+", "")
df_test['title&description'] = df_test['title&description'].str.replace(r"@\S+", "")

CPU times: user 7.18 s, sys: 1.63 s, total: 8.82 s
Wall time: 8.83 s


In [16]:
noise = stopwords.words('russian') + list(punctuation)
tknzr = WordPunctTokenizer()
# morph = MorphAnalyzer()
mystem = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [17]:
def data_preprocessing(text):
    tokens = tknzr.tokenize(text)
    text = [word for word in tokens if (word not in noise and word.isalpha())]
    text = ' '.join(text)
    return text

In [18]:
def lemmatization(text):
    return ''.join(mystem.lemmatize(text))

In [19]:
for idx, text in tqdm(enumerate(df_test['title&description'])):
    df_test.at[idx, 'title&description'] = data_preprocessing(text)

1411348it [04:57, 4736.70it/s]


In [20]:
for idx, text in tqdm(enumerate(df_test['title&description'])):
    df_test.at[idx, 'title&description'] = lemmatization(text)

1411348it [30:26, 772.86it/s]


In [21]:
df_test

,itemid,title&description
0,1778449823,мастерок мастерок фирма форвард белый цвет иде...
1,1677656962,зимний сапог продавать зимний сапог хороший со...
2,1758182804,видеонаблюдение камера комплект atis amd входи...
3,1689811299,запчасть gle запчасть gle\n
4,1804706240,бластер nerf состояние продавать нужный диск п...
...,...,...
1411343,902146704,бетоносмеситель россия бсэ чугун бетономешалка...
1411344,1799983289,разношрокий колесо оригинал porsche panamera д...
1411345,1177237748,комбенз зимний размер комбензон овчина подходи...
1411346,1793055786,смартфон honor отличный состояние использовать...


In [22]:
df_test.to_csv('df_test_preprocessed.csv', index=False)